# ITSM Ticket Similarity - Model Fine-tuning (v6 Refactored)

**Version 6 Refactored** improves upon the original v6 by:
1. **Robust Environment Setup:** Automatically handles NLTK data and library dependencies (Kaggle/Local).
2. **Improved Pipeline:** Cleaner data loading and preprocessing.
3. **Contextual Embeddings:** Retains the structured input format `[Service] [Category] Description`.
4. **Reliable Logging:** Auto-detects the best location for logs.


# 1 . Environment Setup

In [17]:
import os, sys, subprocess, pkg_resources
from pathlib import Path

def ensure_packages(pkgs):
    missing = []
    for name, spec in pkgs.items():
        try:
            pkg_resources.get_distribution(name)
        except pkg_resources.DistributionNotFound:
            missing.append(spec)
    if missing:
        print("📦 Installing:", ", ".join(missing))
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *missing])

def ensure_nltk(resources=("wordnet","omw-1.4","stopwords","punkt")):
    import nltk
    nltk_data = Path.home() / "nltk_data"
    nltk_data.mkdir(exist_ok=True)
    if str(nltk_data) not in nltk.data.path:
        nltk.data.path.append(str(nltk_data))
    for res in resources:
        try:
            nltk.data.find(f"corpora/{res}")
        except LookupError:
            try:
                nltk.data.find(f"tokenizers/{res}")
            except LookupError:
                print(f"⬇️ Downloading NLTK: {res}")
                nltk.download(res, quiet=True, download_dir=str(nltk_data))

    # 4. Pre-download SentenceTransformer model to cache (avoid runtime download delays)
    try:
        from sentence_transformers import SentenceTransformer
        cache_dir = Path(os.environ.get('TRANSFORMERS_CACHE', Path.home() / '.cache' / 'huggingface' / 'hub'))
        cache_dir.mkdir(parents=True, exist_ok=True)
        os.environ['TRANSFORMERS_CACHE'] = str(cache_dir)
        os.environ['SENTENCE_TRANSFORMERS_HOME'] = str(cache_dir)
        print(f'⬇️ Pre-downloading model to cache: sentence-transformers/all-mpnet-base-v2 -> {cache_dir}')
        SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device='cpu')
        print('✅ Model cached.')
    except Exception as e:
        print(f'⚠️ Model pre-download failed: {e}')

def run_setup():
    os.environ['WANDB_DISABLED'] = 'true'
    os.environ['WANDB_MODE'] = 'offline'
    os.environ['WANDB_SILENT'] = 'true'
    os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
    os.environ['TOKENIZERS_PARALLELISM'] = 'false'
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

    pkgs = {
        "sentence-transformers": "sentence-transformers",
        "transformers": "transformers",
        "torch": "torch",               # use Kaggle’s unless missing
        "torchvision": "torchvision",
        "torchaudio": "torchaudio",
        "scikit-learn": "scikit-learn",
        "scipy": "scipy",
        "numpy": "numpy",
        "pandas": "pandas",
        "tqdm": "tqdm",
        "imbalanced-learn": "imbalanced-learn",
        "datasets": "datasets",
        "joblib": "joblib",
        "protobuf": "protobuf<=3.20.1",
        "requests": "requests",
        "python-dotenv": "python-dotenv",
        "openai": "openai",
        "seaborn": "seaborn",
        "matplotlib": "matplotlib",
        "pytorch-lightning": "pytorch-lightning",
    }
    ensure_packages(pkgs)
    ensure_nltk()

run_setup()


2025-11-30 21:28:35,624 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


⬇️ Downloading NLTK: wordnet
⬇️ Downloading NLTK: omw-1.4
⬇️ Pre-downloading model to cache: sentence-transformers/all-mpnet-base-v2 -> /kaggle/working/hf_cache
✅ Model cached.


# 2 Datalogging Setup

In [18]:
# Core Python
import os
import random
import logging
import re
import json
from pathlib import Path
from datetime import datetime
import warnings

# Data Handling
import numpy as np
import pandas as pd

# Progress Bar
from tqdm.auto import tqdm

# ML Frameworks
import torch
from torch.utils.data import DataLoader

# NLP & Metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.stats import pearsonr, spearmanr
from imblearn.over_sampling import SMOTE # Ensure SMOTE is imported for classifier

# Sentence Transformers
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers.evaluation import SentenceEvaluator

# Filter warnings
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Suppress tokenizers warning

# --- Logging Setup ---
def setup_logger():
    # Determine log path: Prefer local current dir, allow agent override
    log_filename = "training_v6_refactored.log"
    log_path = Path.cwd() / log_filename
    
    # If running in Agent env, use agent's temp dir
    if os.environ.get("GEMINI_TEMP_DIR"):
        log_path = Path(os.environ.get("GEMINI_TEMP_DIR")) / log_filename

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_path, mode='w'), # Overwrite for fresh run
            logging.StreamHandler()
        ],
        force=True
    )
    return logging.getLogger(__name__), log_path

logger, LOG_FILE = setup_logger()

def log(msg, level=logging.INFO):
    if level == logging.INFO:
        logger.info(msg)
    elif level == logging.WARNING:
        logger.warning(msg)
    else:
        logger.debug(msg)

log(f"📝 Logging to: {LOG_FILE}")


2025-11-30 21:28:38,057 - INFO - 📝 Logging to: /kaggle/working/training_v6_refactored.log


#3. Model Configuration and Loading

In [19]:
# --- CONFIGURATION ---
CONFIG = {
    'model_name': 'sentence-transformers/all-mpnet-base-v2',
    'output_dir': 'models/v6_refactored_finetuned', # New output directory
    'source_data': 'data/dummy_data_promax.csv', # Will try to resolve this
    'relationship_data': 'data/relationship_pairs.json', # For relationship classifier
    
    # Hyperparameters
    'epochs': 15,
    'batch_size': 32, # Lower batch size for stability
    'lr': 2e-5,
    'max_seq_length': 384,
    
    # Data Strategy
    'num_pairs': 50000, # Number of pairs for training/validation
    'pos_ratio': 0.4,   # 40% Positive, 60% Negative
    'neg_mining_range': (0.2, 0.5), # TF-IDF score range for "Hard Negatives"
    'eval_split': 0.15, # % of data for validation
    
    # Seed
    'seed': 42
}

# Set Seeds
random.seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(CONFIG['seed'])
    log(f"🚀 CUDA Detected: {torch.cuda.get_device_name(0)}")
else:
    log("⚠️ CUDA Not Detected. Running on CPU.")


2025-11-30 21:28:38,120 - INFO - 🚀 CUDA Detected: Tesla P100-PCIE-16GB


In [20]:
def resolve_data_path(path_str):
    """Smart path resolver for Local/Kaggle/Colab/Agent envs."""
    # 1. As-is
    p = Path(path_str)
    if p.exists(): return p.resolve()
    
    # 2. Relative to current script location (for agent or local execution)
    # Using Path.cwd() as a robust base for notebooks
    script_dir = Path.cwd() 
    if (script_dir / path_str).exists(): return (script_dir / path_str).resolve()

    # 3. Common Kaggle/Colab input paths
    # Assuming path_str might be like 'data/file.csv'
    base_filename = Path(path_str).name
    
    kaggle_input_dir = Path("/kaggle/input")
    if kaggle_input_dir.exists():
        for dataset_dir in kaggle_input_dir.iterdir():
            if (dataset_dir / base_filename).exists():
                return (dataset_dir / base_filename).resolve()
            if (dataset_dir / path_str).exists(): # if path_str includes subdir like 'data/'
                return (dataset_dir / path_str).resolve()
                
    colab_dir = Path("/content")
    if (colab_dir / path_str).exists(): return (colab_dir / path_str).resolve()

    raise FileNotFoundError(f"Could not find {path_str} in any common locations (cwd, relative, Kaggle, Colab).")

def load_and_preprocess_data(config):
    source_path = resolve_data_path(config['source_data'])
    log(f"📂 Loading incident data from: {source_path}")
    df = pd.read_csv(source_path)
    
    # Required columns for contextual embedding
    required_cols = ["Number", "Short Description", "Description", "Category", "Subcategory", 
                     "Service", "Service offering", "Assignment group"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Fill NA and Clean Text
    # We ensure all context fields are strings
    placeholders = {"", "nan", "none", "null", "unknown", "n/a", "na"}

    def normalize_field(val: str) -> str:
        s = str(val).strip()
        s = re.sub(r"\s+", " ", s) # Replace multiple spaces with single
        if s.lower() in placeholders:
            return ""
        return s

    for col in [c for c in required_cols if c != "Number"]:
        df[col] = df[col].fillna("").apply(normalize_field)

    # Normalize casing for structured context fields to reduce duplicates
    context_cols = ["Service", "Service offering", "Category", "Subcategory", "Assignment group"]
    for col in context_cols:
        df[col] = df[col].str.lower()

    # Construct Rich Text Representation (Contextual Prefixing)
    # Format: [Service | Service offering] [Category | Subcategory] Group: Assignment group. Short Description. Description
    def build_bracketed(parts):
        clean_parts = [p for p in parts if p]
        return f"[{ ' | '.join(clean_parts) }] " if clean_parts else "" 

    df['context_service'] = df.apply(lambda row: build_bracketed([row['Service'], row['Service offering']]), axis=1)
    df['context_category'] = df.apply(lambda row: build_bracketed([row['Category'], row['Subcategory']]), axis=1)
    df['context_group'] = df.apply(lambda row: f"Group: {row['Assignment group']}. " if row['Assignment group'] else "", axis=1)

    df['text'] = (
        df['context_service'] +
        df['context_category'] +
        df['context_group'] +
        df['Short Description'].str.strip() + ". " +
        df['Description'].str.strip()
    ).str.replace(r"\\s+\\.", ".", regex=True) # Remove space before period
    df['text'] = df['text'].str.replace(r"\\s+", " ", regex=True).str.strip() # Clean up excess spaces

    # Filter empty or too short
    initial_count = len(df)
    min_length = 10 # Configurable if needed
    df = df[df['text'].str.len() >= min_length].copy()
    dropped = initial_count - len(df)
    if dropped > 0:
        log(f"⚠️ Dropped {dropped} incidents due to short/empty text after preprocessing.")
    
    # Create unique group ID for stratified splitting (Category-Subcategory)
    df['category_id'] = df.groupby(['Category', 'Subcategory']).ngroup()
    
    log(f"✅ Preprocessed {len(df)} incidents.")
    log(f"Sample preprocessed text: '{df['text'].iloc[0]}'")
    df = df.reset_index(drop=True)
    log(f'✅ Index reset. Range: {df.index.min()} to {df.index.max()}')
    return df

df_incidents = load_and_preprocess_data(CONFIG)


2025-11-30 21:28:38,197 - INFO - 📂 Loading incident data from: /kaggle/input/itsm-dataset/dummy_data_promax.csv
2025-11-30 21:28:38,838 - INFO - ✅ Preprocessed 10000 incidents.
2025-11-30 21:28:38,839 - INFO - Sample preprocessed text: '[crm (d365, salesforce, genesis, pcube, hussmann services) | bc - basis] [configuration | program bug] Group: piscap l2 workflow. Request: Adjust Configuration/Program bug configuration in CRM (D365, SalesForce, Genesis, PCube, HussMann Services). I encountered an issue where Request: Adjust Configuration/Program bug configuration in CRM (D365, SalesForce, Genesis, PCube, HussMann Services). I'd like assistance to investigate and resolve it.'
2025-11-30 21:28:38,842 - INFO - ✅ Index reset. Range: 0 to 9999


In [21]:
class TextSimilarityCalculator:
    def __init__(self, texts):
        self.lemmatizer = WordNetLemmatizer() if 'wordnet' in nltk.data.path else None
        self.stop_words = set(stopwords.words('english')) if 'stopwords' in nltk.data.path else ENGLISH_STOP_WORDS
        self.vectorizer = TfidfVectorizer(stop_words=list(self.stop_words), max_features=10000)
        
        log("⏳ Fitting TF-IDF for similarity mining...")
        self.tfidf = self.vectorizer.fit_transform(texts)
        log(f"✅ TF-IDF fit complete. Matrix shape: {self.tfidf.shape}")

    def get_tfidf_similarity(self, idx1, idx2):
        if idx1 >= self.tfidf.shape[0] or idx2 >= self.tfidf.shape[0]:
            return 0.0 
        return (self.tfidf[idx1] @ self.tfidf[idx2].T).toarray()[0][0]

def generate_smart_pairs(df, target_count, config):
    """Generates positive and hard negative pairs based on TF-IDF similarity."""
    # Ensure index is reset for direct iloc/loc correspondence
    df = df.reset_index(drop=True)
    
    sim_calculator = TextSimilarityCalculator(df['text'].tolist())

    positive_target = int(target_count * config['pos_ratio'])
    negative_target = target_count - positive_target

    pairs = []
    
    # Group by Category/Subcategory
    # groups indices are now reliable 0..N integers because of reset_index
    groups = df.groupby('category_id').indices 
    valid_groups = {k: list(v) for k, v in groups.items() if len(v) >= 2}
    all_indices = list(df.index)

    log(f"🔎 Generating {positive_target} positive and {negative_target} hard negative pairs...")

    # --- 1. Positive Pairs ---
    pbar_pos = tqdm(total=positive_target, desc="Generating Positives")
    attempts = 0
    while len(pairs) < positive_target and attempts < positive_target * 5:
        attempts += 1
        if not valid_groups: break
        
        gid = random.choice(list(valid_groups.keys()))
        g_idxs = valid_groups[gid] # already a list
        
        if len(g_idxs) < 2: continue
        
        i1, i2 = random.sample(g_idxs, 2)
        
        # i1, i2 are integer positions. Since we reset index, they are also labels.
        # Using iloc is safest for 'text' column access if we mix things up, 
        # but here loc==iloc. We use simple integer indexing for tfidf.
        
        sim = sim_calculator.get_tfidf_similarity(i1, i2)
        
        if sim > 0.3:
            pairs.append(InputExample(texts=[df.at[i1, 'text'], df.at[i2, 'text']], label=1.0))
            pbar_pos.update(1)
            
    # Fill remaining positives
    if len(pairs) < positive_target:
        log(f"⚠️ Filling {positive_target - len(pairs)} remaining positives with random in-group pairs.")
        while len(pairs) < positive_target:
            if not valid_groups: break
            gid = random.choice(list(valid_groups.keys()))
            g_idxs = valid_groups[gid]
            if len(g_idxs) < 2: continue
            i1, i2 = random.sample(g_idxs, 2)
            pairs.append(InputExample(texts=[df.at[i1, 'text'], df.at[i2, 'text']], label=1.0))
            pbar_pos.update(1)
            
    pbar_pos.close()

    # --- 2. Hard Negative Pairs ---
    current_pos_count = len(pairs)
    pbar_neg = tqdm(total=negative_target, desc="Generating Negatives")
    attempts = 0
    max_attempts = negative_target * 10 
    
    while (len(pairs) - current_pos_count) < negative_target and attempts < max_attempts:
        attempts += 1
        
        i1, i2 = random.sample(all_indices, 2)
        
        if df.at[i1, 'category_id'] == df.at[i2, 'category_id']:
            continue
            
        sim = sim_calculator.get_tfidf_similarity(i1, i2)
        
        min_sim, max_sim = config['neg_mining_range']
        if min_sim <= sim <= max_sim:
            pairs.append(InputExample(texts=[df.at[i1, 'text'], df.at[i2, 'text']], label=0.0))
            pbar_neg.update(1)
            
    # Fill remaining negatives
    neg_generated = len(pairs) - current_pos_count
    if neg_generated < negative_target:
         log(f"⚠️ Filling {negative_target - neg_generated} remaining negatives with random cross-category pairs.")
         while (len(pairs) - current_pos_count) < negative_target:
            i1, i2 = random.sample(all_indices, 2)
            if df.at[i1, 'category_id'] != df.at[i2, 'category_id']:
                pairs.append(InputExample(texts=[df.at[i1, 'text'], df.at[i2, 'text']], label=0.0))
                pbar_neg.update(1)
                
    pbar_neg.close()
    
    log(f"✅ Generated {len(pairs)} training pairs.")
    return pairs

# Split incidents into train/eval sets for pair generation
# Note: we split BEFORE pair generation, so we must reset index on the splits individually
train_incidents_df, eval_incidents_df = train_test_split(
    df_incidents,
    test_size=CONFIG['eval_split'],
    stratify=df_incidents['category_id'],
    random_state=CONFIG['seed']
)

log(f"Split Incidents: Train={len(train_incidents_df)}, Eval={len(eval_incidents_df)}")

# Calculate target pairs for each split
train_num_pairs = int(CONFIG['num_pairs'] * (1 - CONFIG['eval_split']))
eval_num_pairs = CONFIG['num_pairs'] - train_num_pairs

train_examples = generate_smart_pairs(train_incidents_df, train_num_pairs, CONFIG)
eval_examples = generate_smart_pairs(eval_incidents_df, eval_num_pairs, CONFIG)

log(f"Final Samples: Training={len(train_examples)}, Evaluation={len(eval_examples)}")



2025-11-30 21:28:38,935 - INFO - Split Incidents: Train=8500, Eval=1500
2025-11-30 21:28:38,938 - INFO - ⏳ Fitting TF-IDF for similarity mining...
2025-11-30 21:28:39,216 - INFO - ✅ TF-IDF fit complete. Matrix shape: (8500, 110)
2025-11-30 21:28:39,218 - INFO - 🔎 Generating 17000 positive and 25500 hard negative pairs...


Generating Positives:   0%|          | 0/17000 [00:00<?, ?it/s]

Generating Negatives:   0%|          | 0/25500 [00:00<?, ?it/s]

2025-11-30 21:29:19,509 - INFO - ✅ Generated 42500 training pairs.
2025-11-30 21:29:19,514 - INFO - ⏳ Fitting TF-IDF for similarity mining...
2025-11-30 21:29:19,566 - INFO - ✅ TF-IDF fit complete. Matrix shape: (1500, 110)
2025-11-30 21:29:19,568 - INFO - 🔎 Generating 3000 positive and 4500 hard negative pairs...


Generating Positives:   0%|          | 0/3000 [00:00<?, ?it/s]

Generating Negatives:   0%|          | 0/4500 [00:00<?, ?it/s]

2025-11-30 21:29:26,851 - INFO - ✅ Generated 7500 training pairs.
2025-11-30 21:29:26,853 - INFO - Final Samples: Training=42500, Evaluation=7500


In [22]:
import torch, os, subprocess
print("cuda?", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    subprocess.run(["nvidia-smi"])

os.environ["TRANSFORMERS_CACHE"] = "/kaggle/working/hf_cache"



cuda? True
Tesla P100-PCIE-16GB
Sun Nov 30 21:29:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P0             34W /  250W |   14279MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------

In [23]:

# --- Model Evaluation Class ---
class ITSMEvaluator(SentenceEvaluator):
    def __init__(self, examples: list[InputExample], batch_size: int = 16, name: str = ''):
        self.examples = examples
        self.batch_size = batch_size
        self.name = name

        self.texts1 = [ex.texts[0] for ex in examples]
        self.texts2 = [ex.texts[1] for ex in examples]
        self.labels = np.array([ex.label for ex in examples])

        self.csv_file = f"{name}_eval_results.csv"
        self.csv_headers = ["epoch", "steps", "spearman", "pearson", "roc_auc", "pr_auc"]

    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) -> float:
        model.eval()
        log(f"📊 Running evaluation at epoch={epoch}, step={steps}...", level=logging.DEBUG)

        # Encode all texts
        embeddings1 = model.encode(self.texts1, batch_size=self.batch_size, show_progress_bar=False, convert_to_numpy=True)
        embeddings2 = model.encode(self.texts2, batch_size=self.batch_size, show_progress_bar=False, convert_to_numpy=True)

        # Calculate cosine similarities
        cosine_scores = np.sum(embeddings1 * embeddings2, axis=1) / (np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1))

        # Calculate metrics
        eval_pearson, _ = pearsonr(self.labels, cosine_scores)
        eval_spearman, _ = spearmanr(self.labels, cosine_scores)

        try:
            roc_auc = roc_auc_score(self.labels, cosine_scores)
            pr_auc = average_precision_score(self.labels, cosine_scores)
        except ValueError:
            roc_auc = 0.0
            pr_auc = 0.0
            log("⚠️ ROC/PR AUC cannot be calculated due to single class in evaluation labels.", level=logging.WARNING)

        log_msg = (
            f"Epoch {epoch if epoch != -1 else 'N/A'} Steps {steps if steps != -1 else 'N/A'}: "
            f"Spearman={eval_spearman:.4f}, Pearson={eval_pearson:.4f}, "
            f"ROC_AUC={roc_auc:.4f}, PR_AUC={pr_auc:.4f}"
        )
        log(log_msg)

        if output_path is not None:
            csv_path = Path(output_path) / self.csv_file
            output_data = [epoch, steps, eval_spearman, eval_pearson, roc_auc, pr_auc]

            if not csv_path.is_file():
                with open(csv_path, 'w', encoding='utf-8') as f:
                    f.write(",".join(self.csv_headers) + "")

            with open(csv_path, 'a', encoding='utf-8') as f:
                f.write(",".join(map(str, output_data)) + "")
        return eval_spearman

# --- Model Setup ---
# Select device (prefer CUDA, then MPS, else CPU)
device = 'cuda' if torch.cuda.is_available() else 'mps' if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available() else 'cpu'
log(f'🔌 Using device: {device}')

model = SentenceTransformer(CONFIG['model_name'], device=device)
model.max_seq_length = CONFIG['max_seq_length']

log(f'✅ Model initialized on device: {device}. Max seq len: {model.max_seq_length}')
pin_memory = device == 'cuda'
num_workers = 2
train_loss = losses.CosineSimilarityLoss(model)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CONFIG['batch_size'], num_workers=num_workers, pin_memory=pin_memory)
log(f'📊 Train batches: {len(train_dataloader)}, Eval examples: {len(eval_examples)}')

evaluator = ITSMEvaluator(eval_examples, batch_size=CONFIG['batch_size'], name='validation')

# --- Training Execution ---
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
save_path = Path(CONFIG['output_dir']) / f"{Path(CONFIG['output_dir']).name}_{timestamp}"
save_path.mkdir(parents=True, exist_ok=True)

log(f"🚀 Starting training... Model will be saved to: {save_path}")
log(f"🚦 Ready to train: batches={len(train_dataloader)}, epochs={CONFIG['epochs']}, batch_size={CONFIG['batch_size']}, device={device}")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=CONFIG['epochs'],
    warmup_steps=int(len(train_dataloader) * CONFIG['epochs'] * 0.1),
    optimizer_params={'lr': CONFIG['lr']},
    output_path=str(save_path),
    evaluation_steps=int(len(train_dataloader) * CONFIG['epochs'] * 0.1),
    save_best_model=True,
    show_progress_bar=True
)
log('model.fit finished')
log('Training complete.')

# --- Final Evaluation ---
log("✨ Reloading best model for final evaluation...")
best_model = SentenceTransformer(str(save_path))

final_evaluator = ITSMEvaluator(eval_examples, batch_size=CONFIG['batch_size'], name='final_evaluation')
final_spearman = final_evaluator(best_model, output_path=str(save_path), epoch='final', steps='final')

log(f"Final Model (best) saved to: {save_path}")


2025-11-30 21:29:27,046 - INFO - 🔌 Using device: cuda
2025-11-30 21:29:27,050 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2025-11-30 21:29:29,665 - INFO - ✅ Model initialized on device: cuda. Max seq len: 384
2025-11-30 21:29:29,666 - INFO - 📊 Train batches: 1329, Eval examples: 7500
2025-11-30 21:29:29,669 - INFO - 🚀 Starting training... Model will be saved to: models/v6_refactored_finetuned/v6_refactored_finetuned_20251130_2129
2025-11-30 21:29:29,669 - INFO - 🚦 Ready to train: batches=1329, epochs=15, batch_size=32, device=cuda
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Evaluator
1329,0.013500,No log,0.848528
1993,0.002900,No log,0.848528
2658,0.000600,No log,0.848528
3986,0.000300,No log,0.848528
3987,0.000300,No log,0.848528


2025-11-30 21:50:11,218 - INFO - Epoch 1.0 Steps 1329: Spearman=0.8485, Pearson=0.9961, ROC_AUC=1.0000, PR_AUC=1.0000
2025-11-30 21:50:11,220 - INFO - Save model to models/v6_refactored_finetuned/v6_refactored_finetuned_20251130_2129
2025-11-30 22:00:58,832 - INFO - Epoch 1.4996237772761476 Steps 1993: Spearman=0.8485, Pearson=0.9986, ROC_AUC=1.0000, PR_AUC=1.0000
2025-11-30 22:00:58,836 - INFO - Save model to models/v6_refactored_finetuned/v6_refactored_finetuned_20251130_2129
2025-11-30 22:11:45,558 - INFO - Epoch 2.0 Steps 2658: Spearman=0.8485, Pearson=0.9995, ROC_AUC=1.0000, PR_AUC=1.0000
2025-11-30 22:32:24,542 - INFO - Epoch 2.999247554552295 Steps 3986: Spearman=0.8485, Pearson=0.9995, ROC_AUC=1.0000, PR_AUC=1.0000
2025-11-30 22:32:24,546 - INFO - Save model to models/v6_refactored_finetuned/v6_refactored_finetuned_20251130_2129
2025-11-30 22:33:20,551 - INFO - Epoch 3.0 Steps 3987: Spearman=0.8485, Pearson=0.9995, ROC_AUC=1.0000, PR_AUC=1.0000
2025-11-30 22:33:20,554 - INFO - 

KeyboardInterrupt: 

In [2]:
# --- Relationship Classifier (Optional) ---
# This part is optional and only runs if imbalanced-learn is available and data exists.

try:
    from imblearn.over_sampling import SMOTE
    IMBLEARN_AVAILABLE = True
except ImportError:
    IMBLEARN_AVAILABLE = False

if IMBLEARN_AVAILABLE:
    rel_data_path = None
    try:
        rel_data_path = resolve_data_path(CONFIG['relationship_data'])
    except FileNotFoundError:
        log(f"⚠️ Relationship data not found at {CONFIG['relationship_data']}. Skipping classifier training.", level=logging.WARNING)

    if rel_data_path and rel_data_path.exists():
        log('🧠 Training Relationship Classifier...')
        with open(rel_data_path, 'r') as f:
            rel_data = json.load(f)

        rel_df = pd.DataFrame(rel_data)
        # Filter valid labels (adjust as per your dataset)
        valid_labels = ['duplicate', 'causal', 'related', 'none']
        rel_df = rel_df[rel_df['label'].isin(valid_labels)]
        log(f"Relationship samples after filtering: {len(rel_df)}")

        if len(rel_df) > 0:
            # Encode features using fine-tuned model
            text_a = rel_df['text_a'].tolist()
            text_b = rel_df['text_b'].tolist()

            log("⏳ Encoding relationship data with the best model...")
            emb_a = best_model.encode(text_a, batch_size=CONFIG['batch_size'], show_progress_bar=False)
            emb_b = best_model.encode(text_b, batch_size=CONFIG['batch_size'], show_progress_bar=False)

            # Feature Engineering: (u, v, |u-v|, u*v)
            X = np.hstack([emb_a, emb_b, np.abs(emb_a - emb_b), emb_a * emb_b])
            y = rel_df['label']

            from sklearn.linear_model import LogisticRegression
            from sklearn.model_selection import train_test_split
            from sklearn.metrics import classification_report
            
            # Only apply SMOTE if there are enough samples and multiple classes
            if len(np.unique(y)) > 1 and len(X) > 1 and len(np.unique(y)) < len(X): # Ensure SMOTE doesn't crash
                smote = SMOTE(k_neighbors=min(2, len(X) - 1), random_state=CONFIG['seed']) # k_neighbors must be <= n_samples-1
                X_res, y_res = smote.fit_resample(X, y)
                log(f"After SMOTE: {len(X_res)} samples")
            else:
                X_res, y_res = X, y
                log("⚠️ Skipping SMOTE due to insufficient samples or single class after filtering.", level=logging.WARNING)

            # Train Classifier
            clf = LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=CONFIG['seed'], solver='lbfgs')
            X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=CONFIG['seed'], stratify=y_res)

            log("⏳ Training Logistic Regression classifier...")
            clf.fit(X_train, y_train)

            # Evaluation
            y_pred = clf.predict(X_test)
            log("✅ Relationship Classifier Report:")
            log(f"\n{classification_report(y_test, y_pred)}")

            # Save Classifier
            import joblib
            classifier_save_path = save_path / "relationship_classifier.joblib"
            joblib.dump(clf, classifier_save_path)
            log(f"✅ Relationship classifier saved to: {classifier_save_path}")
        else:
            log("⚠️ No valid relationship samples to train classifier. Skipping.", level=logging.WARNING)
    else:
        log(f"⚠️ Relationship data not found at resolved path '{rel_data_path}'. Skipping classifier training.", level=logging.WARNING)
else:
    log('⚠️ imbalanced-learn not installed. Skipping relationship classifier.', level=logging.WARNING)


NameError: name 'resolve_data_path' is not defined